In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import rioxarray
import xarray as xr

In [2]:
# data_root = '.'
data_root = 'updated_20250529'

In [3]:
tracks = xr.open_dataset(
    './IBTrACS.NA.v04r00.nc'
)

In [5]:
raft_baseline = xr.open_dataset(
    f'{data_root}/RAFT_historic_baseline.nc'
)
raft_ssp245_near_cold = xr.open_dataset(
    f'{data_root}/RAFT_ssp245_nearFuture_coldModelMean.nc'
)
raft_ssp245_far_cold = xr.open_dataset(
    f'{data_root}/RAFT_ssp245_farFuture_coldModelMean.nc'
)
raft_ssp245_near_hot = xr.open_dataset(
    f'{data_root}/RAFT_ssp245_nearFuture_hotModelMean.nc'
)
raft_ssp245_far_hot = xr.open_dataset(
    f'{data_root}/RAFT_ssp245_farFuture_hotModelMean.nc'
)
raft_ssp585_near_cold = xr.open_dataset(
    f'{data_root}/RAFT_ssp585_nearFuture_coldModelMean.nc'
)
raft_ssp585_far_cold = xr.open_dataset(
    f'{data_root}/RAFT_ssp585_farFuture_coldModelMean.nc'
)
raft_ssp585_near_hot = xr.open_dataset(
    f'{data_root}/RAFT_ssp585_nearFuture_hotModelMean.nc'
)
raft_ssp585_far_hot = xr.open_dataset(
    f'{data_root}/RAFT_ssp585_farFuture_hotModelMean.nc'
)

In [8]:
raft_baseline

<xarray.Dataset>
Dimensions:           (storms: 2337, timesteps: 360)
Dimensions without coordinates: storms, timesteps
Data variables: (12/18)
    vmax_kts          (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    VMPI_t0           (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    U200_t0           (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SHRD_t0           (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    DELV_6            (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    EPTK_t0           (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                ...
    DTL_t0            (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    IBTrACS_vmax      (storms, timesteps) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    datetime_strings  (storms, timesteps) object '' '' '' '' '' ... '' '' '' ''
    year              (storms) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    storm_names       (storms) object '' '' '' '' '' '' '' ... '' '' '' '' '' ''
    storm_ID          (storms) int32 0 1 2 3 4 5 ... 2332 2333 2334 2335 2336
Attributes:
    description:  RAFT baseline simulation TC dataset for North Atlantic basi...

In [6]:
tracks_df = tracks[['name', 'iso_time', 'dist2land']].to_dataframe().reset_index()
tracks_df['time'] = tracks_df.time.astype('datetime64[s]').astype('datetime64[ns]')
tracks_df['over_land'] = (tracks_df.dist2land == 0)

In [15]:
raft_columns = [
    'storm_ID', 'datetime_strings', 'vmax_kts',
    'VMPI_t0', 'U200_t0', 'SHRD_t0', 'DELV_6',
    'LP500_t0', 'PSLV_v3', 'RHLO_t0',
]
raft_df = raft_baseline.to_dataframe().reset_index()[
    raft_columns + ['IBTrACS_vmax']
].rename(columns={
    'IBTrACS_vmax': 'vmax_kts_ibtracs',
}).rename(columns={
    k: f'{k}_baseline' for k in raft_columns[2:]
})
raft_df['datetime_strings'] = pd.to_datetime(raft_df.datetime_strings)
raft_df = raft_df[raft_df.datetime_strings.notna()]

for x, s in [
    (raft_ssp245_near_cold, 'ssp245_near_cold' ),
    (raft_ssp245_far_cold,  'ssp245_far_cold'  ),
    (raft_ssp245_near_hot,  'ssp245_near_hot'  ),
    (raft_ssp245_far_hot,   'ssp245_far_hot'   ),
    (raft_ssp585_near_cold, 'ssp585_near_cold' ),
    (raft_ssp585_far_cold,  'ssp585_far_cold'  ),
    (raft_ssp585_near_hot,  'ssp585_near_hot'  ),
    (raft_ssp585_far_hot,   'ssp585_far_hot'   ),
]:
    df = x.to_dataframe().reset_index()[raft_columns].rename(columns={
        k: f'{k}_{s}' for k in raft_columns[2:]
    })
    df['datetime_strings'] = pd.to_datetime(df.datetime_strings)
    df = df[df.datetime_strings.notna()]
    raft_df = raft_df.merge(
        df,
        how='left',
        on=['storm_ID', 'datetime_strings'],
    )


In [16]:
raft_df

,storm_ID,datetime_strings,vmax_kts_baseline,VMPI_t0_baseline,U200_t0_baseline,SHRD_t0_baseline,DELV_6_baseline,LP500_t0_baseline,PSLV_v3_baseline,RHLO_t0_baseline,...,PSLV_v3_ssp585_near_hot,RHLO_t0_ssp585_near_hot,vmax_kts_ssp585_far_hot,VMPI_t0_ssp585_far_hot,U200_t0_ssp585_far_hot,SHRD_t0_ssp585_far_hot,DELV_6_ssp585_far_hot,LP500_t0_ssp585_far_hot,PSLV_v3_ssp585_far_hot,RHLO_t0_ssp585_far_hot
0,1614,1979-06-11 12:00:00,20.000000,131.209167,111.243896,106.562599,1.348998,0.069586,15.633345,74.071213,...,15.633345,72.941101,20.000000,133.288132,193.112961,209.699661,0.783204,0.069586,15.633345,70.026306
1,1614,1979-06-11 18:00:00,21.348997,135.788773,64.547501,74.705017,1.853827,0.064632,31.074022,72.165245,...,31.074022,71.028267,20.783205,137.867737,146.208511,176.033157,1.170967,0.064632,31.074022,68.164116
2,1614,1979-06-12 00:00:00,23.202826,153.129440,104.456284,102.397980,1.963951,0.092537,15.672303,68.081444,...,15.672303,66.936356,21.954172,156.243103,185.948639,202.329819,1.157271,0.092537,15.672303,64.087013
3,1614,1979-06-12 06:00:00,25.166775,152.578140,123.542870,133.145599,1.804714,0.110185,16.392555,66.792732,...,16.392555,65.661995,23.111443,155.691818,203.787598,231.035324,0.872216,0.110185,16.392555,62.823174
4,1614,1979-06-12 12:00:00,26.971489,152.578140,142.270889,134.235641,1.425113,0.134008,11.313754,64.778915,...,11.313754,63.693298,23.983658,155.691818,220.361008,227.867844,0.479229,0.134008,11.313754,60.918312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17332,2233,2018-11-03 12:00:00,41.901031,66.930473,213.220810,158.530807,1.029509,0.000000,137.659241,84.516556,...,137.659241,82.841698,51.386990,60.894341,309.134155,213.530136,-0.024890,0.000000,137.659241,82.032791
17333,2233,2018-11-03 18:00:00,42.930542,68.977425,212.999710,207.944946,0.872262,0.000000,141.073364,87.871956,...,141.073364,86.303902,51.362103,63.459751,303.308289,259.822052,-0.164957,0.000000,141.073364,85.725273
17334,2233,2018-11-04 00:00:00,43.802803,49.892960,183.957550,224.538071,-0.079326,0.001908,133.319199,89.737549,...,133.319199,88.354958,51.197144,43.947140,269.967438,274.011414,-0.775894,0.001908,133.319199,87.952118
17335,2233,2018-11-04 06:00:00,43.723476,52.936192,169.628387,270.424957,-0.366748,0.025934,174.367249,87.711685,...,174.367249,86.599770,50.421249,47.814629,251.591949,317.486176,-0.852582,0.025934,174.367249,86.361389


In [19]:
storm_ids = []
storm_names = []
storm_labels = []
storm_years = []
storm_files = []
for storm_id in raft_df.storm_ID.unique():

    storm = tracks_df[
        (tracks_df.storm==storm_id) &
        tracks_df.time.notna()
    ][['time', 'lat', 'lon', 'name', 'over_land']].reset_index(drop=True).merge(
        raft_df[
            (raft_df.storm_ID==storm_id)
        ],
        how='left',
        left_on='time',
        right_on='datetime_strings',
    )
    storm_name  = storm.iloc[0]['name'].decode('utf-8').title()
    storm_year  = storm.iloc[0].time.year
    storm_label = f"{storm_name} ({storm_year}) [id:{storm_id}]"
    file_name = f"{storm_name.replace(' ', '_').lower()}_{storm_year}_{storm_id}"
    storm['name'] = storm_name
    storm['year'] = storm_year
    storm['storm_id'] = storm_id
    storm = storm.drop(columns=['datetime_strings', 'storm_ID'])

    storm_ids.append(storm_id)
    storm_names.append(storm_name)
    storm_labels.append(storm_label)
    storm_years.append(storm_year)
    storm_files.append(file_name)

    schema = pa.Schema.from_pandas(storm, preserve_index=False)
    table = pa.Table.from_pandas(storm, preserve_index=False)
    writer = pa.ipc.new_file(f'../static/data/{file_name}.arrow', schema)
    writer.write(table)
    writer.close()


In [20]:
storm_reference = pd.DataFrame({
    'id': storm_ids,
    'name': storm_names,
    'year': storm_years,
    'label': storm_labels,
    'value': storm_files,
})

In [21]:
storm_reference.to_json('../src/lib/storms.js', orient='records', indent=2)